# Predicting Airline Delay Project
## Part 2: Adding the weather data
In this part, we load the entire airline dataset, clean the data and make a subset of the data containing flights originating and ending in the 50 busiest airports. Then we append the weather data for origin and destination airports at the time of takeoff (CSR_DEP_TIME) and landing (CSR_ARR_TIME). 

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('Full_airline_data.csv')

## To join with our weather data, the top 50 airports (sorted by air traffic) are selected.
** It is observed that ~62% of all flights originate and end in the top 50 airports

In [4]:
#Make subset of data containing only flights from top 50 airports
top50_airport = pd.read_csv('top50airports.csv')['IATA'].tolist()
df_sub = df[df['ORIGIN'].isin(top50_airport)]
df_sub = df_sub[df_sub['DEST'].isin(top50_airport)]

In [5]:
#Check the % of data retained:
data_retained = len(df_sub.index)/len(df.index)
print('Data Retained: '+str(round(data_retained*100,2))+' %')

Data Retained: 62.34 %


## Join the Weather dataset obtained from www.ncdc.noaa.gov.in:

In [ ]:
#Read the weather csv files
tic = time.time()
#Read all the airline data files
df_weather = pd.DataFrame()

f = []
for (dirpath,dirnames,filenames) in walk('weather/'):
    f.extend(filenames)
    break

for file in f[1:]:
    df_weather = df_weather.append(pd.read_csv('weather/'+file,low_memory=False))
    
toc = time.time()
print(toc-tic)

In [ ]:
#Select the weather parameters which affect flight status: Visibility, Temperature, Wind Speed, Precipitation
df_weather = df_weather[['STATION_NAME','DATE','HOURLYVISIBILITY','HOURLYDRYBULBTEMPC','HOURLYWindSpeed','HOURLYPrecip']].copy()

## Preparing the Weather dataset:
* Replace Long format station name with IATA codes #Need to fix an automated way to do this
* Fix incorrect and erroneous data, such as characters in temperature fields, etc
* Convert Timestamp into YEAR, MONTH, DAY_OF_MONTH and HOUR
* Remove duplicates from the dataset, i.e. multiple entries from same STATION for same HOUR on a particular Date
* Replace NaN with 0 in HOURLY_PRECIP
* Replace NaN with Mean Visibility in HOURLYVISIBILITY

In [ ]:
#Replacing Long Format Station Names with IATA Airport codes
df_weather['STATION_NAME'].replace('ATLANTA HARTSFIELD INTERNATIONAL AIRPORT GA US','ATL',inplace=True)
df_weather['STATION_NAME'].replace('CHICAGO OHARE INTERNATIONAL AIRPORT IL US','ORD',inplace=True)
df_weather['STATION_NAME'].replace('DAL FTW WSCMO AIRPORT TX US','DFW',inplace=True)
df_weather['STATION_NAME'].replace('DENVER INTERNATIONAL AIRPORT CO US','DEN',inplace=True)
df_weather['STATION_NAME'].replace('LOS ANGELES INTERNATIONAL AIRPORT CA US','LAX',inplace=True)
df_weather['STATION_NAME'].replace('SAN FRANCISCO INTERNATIONAL AIRPORT CA US','SFO',inplace=True)
df_weather['STATION_NAME'].replace('PHOENIX SKY HARBOR INTERNATIONAL AIRPORT AZ US','PHX',inplace=True)
df_weather['STATION_NAME'].replace('HOUSTON INTERCONTINENTAL AIRPORT TX US','IAH',inplace=True)
df_weather['STATION_NAME'].replace('LAS VEGAS MCCARRAN INTERNATIONAL AIRPORT NV US','LAS',inplace=True)
df_weather['STATION_NAME'].replace('MINNEAPOLIS ST PAUL INTERNATIONAL AIRPORT MN US','MSP',inplace=True)
df_weather['STATION_NAME'].replace('DETROIT METROPOLITAN AIRPORT MI US','DTW',inplace=True)
df_weather['STATION_NAME'].replace('SEATTLE TACOMA INTERNATIONAL AIRPORT WA US','SEA',inplace=True)
df_weather['STATION_NAME'].replace('ORLANDO INTERNATIONAL AIRPORT FL US','MCO',inplace=True)
df_weather['STATION_NAME'].replace('BOSTON MA US','BOS',inplace=True)
df_weather['STATION_NAME'].replace('CHARLOTTE DOUGLAS AIRPORT NC US','CLT',inplace=True)
df_weather['STATION_NAME'].replace('NEWARK LIBERTY INTERNATIONAL AIRPORT NJ US','EWR',inplace=True)
df_weather['STATION_NAME'].replace('SALT LAKE CITY INTERNATIONAL AIRPORT UT US','SLC',inplace=True)
df_weather['STATION_NAME'].replace('LA GUARDIA AIRPORT NY US','LGA',inplace=True)
df_weather['STATION_NAME'].replace('JFK INTERNATIONAL AIRPORT NY US','JFK',inplace=True)
df_weather['STATION_NAME'].replace('BALTIMORE WASHINGTON INTERNATIONAL AIRPORT MD US','BWI',inplace=True)
df_weather['STATION_NAME'].replace('CHICAGO MIDWAY AIRPORT IL US','MDW',inplace=True)
df_weather['STATION_NAME'].replace('MIAMI INTERNATIONAL AIRPORT FL US','MIA',inplace=True)
df_weather['STATION_NAME'].replace('SAN DIEGO INTERNATIONAL AIRPORT CA US','SAN',inplace=True)
df_weather['STATION_NAME'].replace('WASHINGTON REAGAN NATIONAL AIRPORT VA US','DCA',inplace=True)
df_weather['STATION_NAME'].replace('FORT LAUDERDALE HOLLYWOOD INTERNATIONAL AIRPORT FL US','FLL',inplace=True)
df_weather['STATION_NAME'].replace('PHILADELPHIA INTERNATIONAL AIRPORT PA US','PHL',inplace=True)
df_weather['STATION_NAME'].replace('TAMPA INTERNATIONAL AIRPORT FL US','TPA',inplace=True)
df_weather['STATION_NAME'].replace('DALLAS FAA AIRPORT TX US','DAL',inplace=True)
df_weather['STATION_NAME'].replace('HOUSTON WILLIAM P HOBBY AIRPORT TX US','HOU',inplace=True)
df_weather['STATION_NAME'].replace('PORTLAND INTERNATIONAL AIRPORT OR US','PDX',inplace=True)
df_weather['STATION_NAME'].replace('NASHVILLE INTERNATIONAL AIRPORT TN US','BNA',inplace=True)
df_weather['STATION_NAME'].replace('ST LOUIS LAMBERT INTERNATIONAL AIRPORT MO US','STL',inplace=True)
df_weather['STATION_NAME'].replace('WASHINGTON DULLES INTERNATIONAL AIRPORT VA US','IAD',inplace=True)
df_weather['STATION_NAME'].replace('HONOLULU INTERNATIONAL AIRPORT HI US','HNL',inplace=True)
df_weather['STATION_NAME'].replace('OAKLAND METROPOLITAN INTERNATIONAL AIRPORT CA US','OAK',inplace=True)
df_weather['STATION_NAME'].replace('AUSTIN BERGSTROM INTERNATIONAL AIRPORT TX US','AUS',inplace=True)
df_weather['STATION_NAME'].replace('KANSAS CITY INTERNATIONAL AIRPORT MO US','MCI',inplace=True)
df_weather['STATION_NAME'].replace('NEW ORLEANS INTERNATIONAL AIRPORT LA US','MSY',inplace=True)
df_weather['STATION_NAME'].replace('SAN JOSE CA US','SJC',inplace=True)
df_weather['STATION_NAME'].replace('SACRAMENTO METROPOLITAN AIRPORT CA US','SMF',inplace=True)
df_weather['STATION_NAME'].replace('SANTA ANA JOHN WAYNE AIRPORT CA US','SNA',inplace=True)
df_weather['STATION_NAME'].replace('CLEVELAND HOPKINS INTERNATIONAL AIRPORT OH US','CLE',inplace=True)
df_weather['STATION_NAME'].replace('RALEIGH AIRPORT NC US','RDU',inplace=True)
df_weather['STATION_NAME'].replace('MILWAUKEE MITCHELL INTERNATIONAL AIRPORT WI US','MKE',inplace=True)
df_weather['STATION_NAME'].replace('SAN ANTONIO INTERNATIONAL AIRPORT TX US','SAT',inplace=True)
df_weather['STATION_NAME'].replace('INDIANAPOLIS INTERNATIONAL AIRPORT IN US','IND',inplace=True)
df_weather['STATION_NAME'].replace('FORT MYERS SW FLORIDA REGIONAL AIRPORT FL US','RSW',inplace=True)
df_weather['STATION_NAME'].replace('PITTSBURGH ASOS PA US','PIT',inplace=True)
df_weather['STATION_NAME'].replace('SAN JUAN L M MARIN INTERNATIONAL AIRPORT US','SJU',inplace=True)
df_weather['STATION_NAME'].replace('PORT COLUMBUS INTERNATIONAL AIRPORT OH US','CMH',inplace=True)

In [ ]:
def tryconvert(x):
        try:
            if str(x)[-1].isalpha():
                return(float(str(x)[:-1]))
            else:
                return(float(str(x)))
        except:
            return(np.nan)

#Get the Year, Month, Date and Hour from df_weather
#Delete duplicate rows with data from the same station and same date/hour
#Delete "DATE"

df_weather['DATE'] = pd.to_datetime(df_weather['DATE'])

df_weather['YEAR']= df_weather['DATE'].apply(lambda time: time.year)
df_weather['MONTH']= df_weather['DATE'].apply(lambda time: time.month)
df_weather['DAY_OF_MONTH']= df_weather['DATE'].apply(lambda time: time.day)
df_weather['HOUR']= df_weather['DATE'].apply(lambda time: time.hour)

df_weather.drop_duplicates(['STATION_NAME','YEAR','MONTH','DAY_OF_MONTH','HOUR'],inplace=True)
df_weather.drop('DATE',axis = 1,inplace=True)

df_weather['HOURLYVISIBILITY'] = df_weather['HOURLYVISIBILITY'].apply(lambda x: tryconvert(x))
df_weather['HOURLYDRYBULBTEMPC'] = df_weather['HOURLYDRYBULBTEMPC'].apply(lambda x: tryconvert(x))
df_weather['HOURLYWindSpeed'] = df_weather['HOURLYWindSpeed'].apply(lambda x: tryconvert(x))
df_weather['HOURLYPrecip'] = df_weather['HOURLYPrecip'].apply(lambda x: tryconvert(x))

#Replace NaNs with 0 in Hourly Precip
df_weather['HOURLYPrecip'].fillna(value=0,inplace=True)
#Replace NaNs with average value for Hourly Visibility
df_weather['HOURLYVISIBILITY'].fillna(df_weather['HOURLYVISIBILITY'].mean(),inplace=True)

In [ ]:
#Check for any missing data:
#Missing data will show up as Yellow lines
sns.heatmap(df_weather.isnull(),yticklabels=False,cbar=False,cmap='viridis')

## Calculate Average Weather values for each station
For every weather station, we calculate the average or mean weather parameters, i.e. Annual Mean Temperature, Annual Mean Precipitation, Annual Mean Visibility etc.
Two Dataframes are created for simplicity - One for Origin, and one for Destination. They are the same dataframes, except for the Column Names. 

In [ ]:
df_avg_DEP = df_weather.groupby('STATION_NAME').mean()
df_avg_DEP.drop(['YEAR','MONTH','DAY_OF_MONTH','HOUR'],axis = 1,inplace=True)
df_avg_DEP.reset_index(drop=False,inplace=True)
df_avg_DEP.rename(index=str, columns={"STATION_NAME": "ORIGIN"},inplace=True)
df_avg_DEP.rename(index=str, columns={"HOURLYVISIBILITY": "DEP_AVG_HOURLYVISIBILITY"},inplace=True)
df_avg_DEP.rename(index=str, columns={"HOURLYDRYBULBTEMPC": "DEP_AVG_HOURLYDRYBULBTEMPC"},inplace=True)
df_avg_DEP.rename(index=str, columns={"HOURLYWindSpeed": "DEP_AVG_HOURLYWindSpeed"},inplace=True)
df_avg_DEP.rename(index=str, columns={"HOURLYPrecip": "DEP_AVG_HOURLYPrecip"},inplace=True)

df_avg_ARR = df_weather.groupby('STATION_NAME').mean()
df_avg_ARR.drop(['YEAR','MONTH','DAY_OF_MONTH','HOUR'],axis = 1,inplace=True)
df_avg_ARR.reset_index(drop=False,inplace=True)
df_avg_ARR.rename(index=str, columns={"STATION_NAME": "DEST"},inplace=True)
df_avg_ARR.rename(index=str, columns={"HOURLYVISIBILITY": "ARR_AVG_HOURLYVISIBILITY"},inplace=True)
df_avg_ARR.rename(index=str, columns={"HOURLYDRYBULBTEMPC": "ARR_AVG_HOURLYDRYBULBTEMPC"},inplace=True)
df_avg_ARR.rename(index=str, columns={"HOURLYWindSpeed": "ARR_AVG_HOURLYWindSpeed"},inplace=True)
df_avg_ARR.rename(index=str, columns={"HOURLYPrecip": "ARR_AVG_HOURLYPrecip"},inplace=True)

In [ ]:
#Create two copies of the Weather Dataframe, so that we can two Joins, one for Origin, and one for Destination
df_weather_origin = df_weather.copy()
df_weather_dest = df_weather.copy()
del df_weather

#Rename the Columns, add DEP_ to each column name and STATION_NAME to ORIGIN
df_weather_origin.rename(index=str, columns={"STATION_NAME": "ORIGIN"},inplace=True)
df_weather_origin.rename(index=str, columns={"HOURLYVISIBILITY": "DEP_HOURLYVISIBILITY"},inplace=True)
df_weather_origin.rename(index=str, columns={"HOURLYDRYBULBTEMPC": "DEP_HOURLYDRYBULBTEMPC"},inplace=True)
df_weather_origin.rename(index=str, columns={"HOURLYWindSpeed": "DEP_HOURLYWindSpeed"},inplace=True)
df_weather_origin.rename(index=str, columns={"HOURLYPrecip": "DEP_HOURLYPrecip"},inplace=True)
df_weather_origin.rename(index=str, columns={"HOUR": "DEP_HOUR"},inplace=True)

#Rename the Columns, add ARR_ to each column name and STATION_NAME to DEST
df_weather_dest.rename(index=str, columns={"STATION_NAME": "DEST"},inplace=True)
df_weather_dest.rename(index=str, columns={"HOURLYVISIBILITY": "ARR_HOURLYVISIBILITY"},inplace=True)
df_weather_dest.rename(index=str, columns={"HOURLYDRYBULBTEMPC": "ARR_HOURLYDRYBULBTEMPC"},inplace=True)
df_weather_dest.rename(index=str, columns={"HOURLYWindSpeed": "ARR_HOURLYWindSpeed"},inplace=True)
df_weather_dest.rename(index=str, columns={"HOURLYPrecip": "ARR_HOURLYPrecip"},inplace=True)
df_weather_dest.rename(index=str, columns={"HOUR": "ARR_HOUR"},inplace=True)

## Join the Actual Weather and Average Weather for Origin and Destination Airports for each flight in the df_sub DataFrame

In [ ]:
#Merge the Actual Weather during Take-off and Landing for each flight 
df_sub = pd.merge(df_sub, df_weather_origin, on=['ORIGIN','YEAR','MONTH','DAY_OF_MONTH','DEP_HOUR'], how='left')
df_sub = pd.merge(df_sub, df_weather_dest, on=['DEST','YEAR','MONTH','DAY_OF_MONTH','ARR_HOUR'], how='left')
 

In [ ]:
#Merge the Average weather for Origin and Destination Airports for each flight:
df_sub = pd.merge(df_sub,df_avg_DEP,how='left',on='ORIGIN')
df_sub = pd.merge(df_sub,df_avg_ARR,how='left',on='DEST')

## Save both DataFrames as .csv files

In [ ]:
tic = time.time()
df_sub.to_csv('Airline+Weather_data.csv',index=False)
toc = time.time()
print(toc-tic)

In [ ]:
#Create a randomly selected smaller dataset for testing purposes
df2 = df_sub.loc[np.random.choice(df.index, 1000000, replace = False)]
df2.to_csv('medium_data.csv',index=False)